Imports

In [49]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd
import requests
import re

Opening The Url

In [2]:
url = urlopen("http://www.northsouth.edu/")
nsu = soup(url.read())
#Examples of how to find by tag
#links = nsu.find_all('a')
#scripts = nsu.find_all('script')

Going through all the links

In [3]:
links = []

for link in nsu.find_all('a'):
  if 'href' in link.attrs:
    links.append((link.attrs['href']))

Creating dataframe for the links

In [4]:
links_df = pd.DataFrame(links)
links_df.head(2)

#Nameing the columns
links_df.columns = ['links']
links_df.head(2)


,links
0,contact-us.html
1,archive.html


Modifying the dataframe

In [5]:
# Removing empty links from the dataframe
links_df = links_df[links_df["links"].str.contains("#") == False]
links_df.head(2)

,links
0,contact-us.html
1,archive.html


In [6]:
# Removing external links from the data frame
internal_links = links_df[links_df["links"].str.contains("http") == False]
internal_links.head(2)

,links
0,contact-us.html
1,archive.html


In [7]:
# Adding Https infront of all the addresses
internal_links['links'] = 'http://www.northsouth.edu/' + internal_links['links'].astype(str)
internal_links.head(2)

/var/folders/_w/kwxvx6js1gvfr3r6sz47fpmc0000gn/T/ipykernel_26261/3951678606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  internal_links['links'] = 'http://www.northsouth.edu/' + internal_links['links'].astype(str)


,links
0,http://www.northsouth.edu/contact-us.html
1,http://www.northsouth.edu/archive.html


In [8]:
# Creating new dataframe for external links
external_links = links_df[links_df["links"].str.contains("http") == True]
external_links.head(2)

,links
5,http://library.northsouth.edu
7,http://institutions.northsouth.edu/oea/interna...


Getting all the text data from the homepage

In [59]:
for data in nsu(['style', 'script', 'a', 'footer', 'nav']):
  # Remove tags
  data.decompose()

data = nsu.get_text()
data = data.replace("\n", " ")
#data = nsu.findAll('p')
data = re.sub(' +',' ',data)

data

' Home | North South University NSU Ranked #1 In Bangladesh with the following international university ranking agencies 1. Times Higher Education (THE): 2023 (Public/Private Universities) 2. QS World University Ranking: 2022, 2021, 2020 (Private) 3. QS Asia University Ranking: 2023, 2022, 2021, 2020, 2019 (Private) 4. Webometrics University Ranking: 2022 (Private) Undergraduate Study Graduate Study Research Notice 27/03/2023 14/03/2023 13/03/2023 02/03/2023 23/02/2023 '

Creating a new list for all data of different website pages

In [60]:
website_data = pd.DataFrame({'address': ['http://www.northsouth.edu/'],
                             'title': ['Home | North South University'],
                             'data': [data]})
#website_data[data].replace("\n"," ")
website_data

,address,title,data
0,http://www.northsouth.edu/,Home | North South University,Home | North South University NSU Ranked #1 I...


Interlink and External Link seperated into their own list

In [11]:
internal_link_list = []
external_link_list = []

for item in links:
  if "http" in item:
    external_link_list.append(item)
  elif "#" in item:
    continue
  elif "pdf" in item:
    continue
  else:
    internal_link_list.append(item)


Adding full address to all internal links

In [12]:
internal_link_list_full = []
for item in internal_link_list:
  temp = "http://www.northsouth.edu/" + item
  internal_link_list_full.append(temp)

print(len(internal_link_list_full))
internal_link_list_full

95


['http://www.northsouth.edu/contact-us.html',
 'http://www.northsouth.edu/archive.html',
 'http://www.northsouth.edu/newsletter/nsu-catalog.html',
 'http://www.northsouth.edu/journals/',
 'http://www.northsouth.edu/newsletter.html',
 'http://www.northsouth.edu/academic/academic-calendar/',
 'http://www.northsouth.edu/about/brief-history.html',
 'http://www.northsouth.edu/about/vms.html',
 'http://www.northsouth.edu/about/int-ab.html',
 'http://www.northsouth.edu/about/accreditation.html',
 'http://www.northsouth.edu/about/international-recognition.html',
 'http://www.northsouth.edu/about/facts.html',
 'http://www.northsouth.edu/about/bot/',
 'http://www.northsouth.edu/academic/sbe/',
 'http://www.northsouth.edu/academic/sbe/acc-fin/',
 'http://www.northsouth.edu/academic/sbe/economics.html',
 'http://www.northsouth.edu/academic/sbe/mgt.html',
 'http://www.northsouth.edu/academic/sbe/mib.html',
 'http://www.northsouth.edu/academic/sbe/mba-emba/',
 'http://www.northsouth.edu/academic/sep

Getting All Links

In [13]:
def get_links_from_list(url_list, temp_url_list):
  for url in url_list:
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    #print("Accessing URL -------------------------- " + url)
    nsu_data = soup(response.text, 'lxml')
    for link in nsu_data.find_all('a'):
      if 'href' in link.attrs:
          temp = link.attrs['href']
          #print("Url Found - " + temp)
          if "http" in temp:
            continue
          elif "#" in temp:
            continue
          elif "pdf" in temp:
              continue
          elif "//" in temp:
            continue
          elif "gmail" in temp:
            continue
          elif "mailto" in temp:
            continue
          elif "jpg" in temp:
            continue
          elif "jpeg" in temp:
            continue
          elif "png" in temp:
            continue
          elif "doc" in temp:
            continue
          elif "docx" in temp:
            continue
          elif "xls" in temp:
            continue
          elif "pptx" in temp:
            continue
          elif "~" in temp:
            continue
          elif "http://www.northsouth.edu/" + temp in temp_url_list:
            continue
          else:
            #print("Appending Link ----- http://www.northsouth.edu/" +temp)
            temp_url_list.append("http://www.northsouth.edu/" + temp)
  print("def get links from list temp list size")
  print(len(temp_url_list))

In [14]:
# Getting links from the links in internal links list
temp_url_list_1 = []
temp_url_list_2 = []

print(len(temp_url_list_1))
print(len(temp_url_list_2))

get_links_from_list(internal_link_list_full, temp_url_list_1)
get_links_from_list(temp_url_list_1, temp_url_list_2)

0
0
def get links from list temp list size
735
def get links from list temp list size
2005


In [15]:
print(len(temp_url_list_1))
print(len(temp_url_list_2))

735
2005


In [16]:
print(temp_url_list_2)

['http://www.northsouth.edu/academic/academic-calendar/', 'http://www.northsouth.edu/newsletter.html', 'http://www.northsouth.edu/journals/', 'http://www.northsouth.edu/newsletter/nsu-catalog.html', 'http://www.northsouth.edu/all-career-at-nsu/', 'http://www.northsouth.edu/archive.html', 'http://www.northsouth.edu/contact-us.html', 'http://www.northsouth.edu/about/brief-history.html', 'http://www.northsouth.edu/about/vms.html', 'http://www.northsouth.edu/about/int-ab.html', 'http://www.northsouth.edu/about/accreditation.html', 'http://www.northsouth.edu/about/international-recognition.html', 'http://www.northsouth.edu/about/facts.html', 'http://www.northsouth.edu/about/bot/', 'http://www.northsouth.edu/academic/sbe/', 'http://www.northsouth.edu/academic/sbe/acc-fin/', 'http://www.northsouth.edu/academic/sbe/economics.html', 'http://www.northsouth.edu/academic/sbe/mgt.html', 'http://www.northsouth.edu/academic/sbe/mib.html', 'http://www.northsouth.edu/academic/sbe/mba-emba/', 'http://ww

In [17]:
temp_url_list_1

['http://www.northsouth.edu/academic/academic-calendar/',
 'http://www.northsouth.edu/newsletter.html',
 'http://www.northsouth.edu/journals/',
 'http://www.northsouth.edu/newsletter/nsu-catalog.html',
 'http://www.northsouth.edu/all-career-at-nsu/',
 'http://www.northsouth.edu/archive.html',
 'http://www.northsouth.edu/contact-us.html',
 'http://www.northsouth.edu/about/brief-history.html',
 'http://www.northsouth.edu/about/vms.html',
 'http://www.northsouth.edu/about/int-ab.html',
 'http://www.northsouth.edu/about/accreditation.html',
 'http://www.northsouth.edu/about/international-recognition.html',
 'http://www.northsouth.edu/about/facts.html',
 'http://www.northsouth.edu/about/bot/',
 'http://www.northsouth.edu/academic/sbe/',
 'http://www.northsouth.edu/academic/sbe/acc-fin/',
 'http://www.northsouth.edu/academic/sbe/economics.html',
 'http://www.northsouth.edu/academic/sbe/mgt.html',
 'http://www.northsouth.edu/academic/sbe/mib.html',
 'http://www.northsouth.edu/academic/sbe/mba

In [67]:
for tag in internal_link_list_full:
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(tag, headers=headers)
    print("Accessing URL -------------------------- " + tag)
    data = soup(response.text, 'lxml')
    
    if data.find('title') is None:
        continue
    elif data.find('title').string is None:
        continue
    else:
        title = data.find('title').string
    
        for item in data(['style', 'script', 'a', 'footer', 'nav', 'title']):
            # Remove tags
            item.decompose()
        
        #print(title)
    
        temp_data = data.get_text()
        temp_data = temp_data.replace("\n", " ")
        temp_data = re.sub(' +',' ',temp_data)
        print(temp_data)
    
        temp_df = pd.DataFrame({'address': [tag],
                                'title': [title],
                                'data': [temp_data]})
        
        #pd.concat([website_data, temp_df], ignore_index=True)
        website_data = website_data.append({'address': tag, 'title': title, 'data': temp_data}, ignore_index=True)
        #website_data = website_data.append({'title': title}, ignore_index=True)
        #website_data = website_data.append({'data': temp_data}, ignore_index=True)

website_data
    

Accessing URL -------------------------- http://www.northsouth.edu/contact-us.html
 registrar@northsouth.edu Location Map 8202he First Private University in Bangladesh Contact North South University


/var/folders/_w/kwxvx6js1gvfr3r6sz47fpmc0000gn/T/ipykernel_26261/4273083322.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  website_data = website_data.append({'address': tag, 'title': title, 'data': temp_data}, ignore_index=True)


Accessing URL -------------------------- http://www.northsouth.edu/archive.html
 Center of Excellence in Higher Education The First Private University in Bangladesh Archive 2023 2022 2021 2020 2019 2018 2017 2016 


/var/folders/_w/kwxvx6js1gvfr3r6sz47fpmc0000gn/T/ipykernel_26261/4273083322.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  website_data = website_data.append({'address': tag, 'title': title, 'data': temp_data}, ignore_index=True)


Accessing URL -------------------------- http://www.northsouth.edu/newsletter/nsu-catalog.html
 Center of Excellence in Higher Education The First Private University in Bangladesh Catalogs Year Action 2015-2016 2014-2015 


...

Accessing URL -------------------------- http://www.northsouth.edu/faculty-members/shss/pss/
 Center of Excellence in Higher Education The First Private University in Bangladesh Department of Political Science & Sociology - Faculty Members Professor & Dean of SHSS Ph.D., University of Kent at Canterbury (UKC), UK 	 Professor & Chair, Department of History and Philosophy Ph.D., La Trobe University, Australia MA University of Waterloo, Canada 	 Professor, Chair & Director, SIPG Program Ph. D., University of Dhaka, Bangladesh M. Phil., University of Bergen, Norway 	 Professor Ph. D., Clark Atlanta University, Atlanta, USA 	 Professor Ph.D., University of Manitoba, CanadaM.Sc., Asian Institute of Technology, Thailand 	 Professor Ph.D., University of Alberta, CA 	 Associate Professor and Director, Confucius Institute at NSU Ph.D., Cardiff University, UKM.A., University of Nottingham , UK 	 Associate Professor PhD, Osaka University, Japan 	 Associate Professor Ph.D., The University of New En

/var/folders/_w/kwxvx6js1gvfr3r6sz47fpmc0000gn/T/ipykernel_26261/4273083322.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  website_data = website_data.append({'address': tag, 'title': title, 'data': temp_data}, ignore_index=True)


Accessing URL -------------------------- http://www.northsouth.edu/faculty-members/shss/law/
 Center of Excellence in Higher Education The First Private University in Bangladesh Department of Law - Faculty Members Professor Ph.D., Macquarie University LL.M., National University of Singapore, LL.B. (Hons), Dhaka Uni 	 Associate Professor Ph. D., University of Wollongong, Australia Associate Professor J.S.D., University of Maine School of Law, USA, LL.M., University of London in Association with Queen Mary & UCL LL.B., University of London, UK Barrister-At-Law (U.K) 	 Senior Lecturer LLM., University of Connecticut, USA LLM., Univeristy of Montreal, Candada 	 Senior Lecturer & Chair M.A.- King’s College London, UK L.L.B- University of Essex, UK 	 Senior Lecturer LL.M, University of Edinburgh, UK 	 Senior Lecturer LLM, National Academy of Legal Studies and Research (NALSAR), India LL.B (Honours) BRAC University 	 Senior Lecturer LL.M., University of Warwick, UK 	 Lecturer LL.M., Universit

/var/folders/_w/kwxvx6js1gvfr3r6sz47fpmc0000gn/T/ipykernel_26261/4273083322.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  website_data = website_data.append({'address': tag, 'title': title, 'data': temp_data}, ignore_index=True)


KeyboardInterrupt: 

In [65]:
print(website_data.data)

0     Home | North South University NSU Ranked #1 I...
1     Center of Excellence in Higher Education The ...
2     Center of Excellence in Higher Education The ...
3     Center of Excellence in Higher Education The ...
4     Center of Excellence in Higher Education The ...
5     Center of Excellence in Higher Education The ...
6     Center of Excellence in Higher Education The ...
7     Center of Excellence in Higher Education The ...
8     Center of Excellence in Higher Education The ...
9     Center of Excellence in Higher Education The ...
Name: data, dtype: object
